In [9]:
import pandas as pd
import calendar

#Calculate the efficiency score
def calculate_efficiency_score_row(row):
    if row['Duration'] < row['Expected Duration']:
        return float(format((row['Expected Duration'] - row['Duration']) / row['Expected Duration'], '.3f'))
    else:
        return float(format(((row['Duration'] - row['Expected Duration']) / row['Expected Duration'] * (-1)), '.3f'))
    
def calculate_performance_score(efficiency, rating, success_rate):

    performance_score = format(((efficiency * 100 * 0.4 * 2) + (rating * 20 * 0.3) + (success_rate * 100 * 0.3)), '.3f')

    return float(performance_score)

def performance_score_sheet(dummy_data, efficiency_data):

    dummy_data['Rating'] = pd.to_numeric(dummy_data['Rating'], errors='coerce')
    dummy_data['Success'] = pd.to_numeric(dummy_data['Success'], errors='coerce')

    grouped_data = dummy_data.groupby(['Dentist Name', 'Treatment']).agg({
        'Rating': 'mean',
        'Success': 'mean'
    }).reset_index()

    grouped_data = grouped_data.merge(efficiency_data.melt(id_vars='Dentist Name', var_name='Treatment', value_name='Efficiency'), on=['Dentist Name', 'Treatment'], how='left')

    grouped_data['Performance Score'] = grouped_data.apply(
        lambda row: calculate_performance_score(row['Efficiency'], row['Rating'], row['Success']), axis=1)

    performance_df = grouped_data.pivot(index='Dentist Name', columns='Treatment', values='Performance Score').reset_index()
    performance_df = performance_df.fillna(0)

    return performance_df



def get_performance_metrics(dentist_name, treatment, df, df2):
    target_dentist = df[df['Dentist Name'] == dentist_name]
    target_treatment = target_dentist[target_dentist['Treatment'] == treatment]

    target_success = target_treatment[target_treatment['Success'] == 1]

    target_expected_duration = df2[df2['Treatment Menu Names'] == treatment]
    target_expected_duration = target_expected_duration['Duration'].values[0]

    count = target_treatment.value_counts().sum()
    average = format(target_treatment['Duration'].mean(), '.3f')
    max = format(target_treatment['Duration'].max(), '.3f')
    min = format(target_treatment['Duration'].min(), '.3f')
    std = format(target_treatment['Duration'].std(), '.3f')
    rating_avg = float(format(target_treatment['Rating'].mean(), '.1f'))
    success_rate = float(target_success['Success'].value_counts().sum())/float(count)

   # Calculate the performance score
    if float(average) < float(target_expected_duration):
        efficiency = (target_expected_duration - float(average)) / target_expected_duration * 1
    else:
        efficiency = (float(average) - target_expected_duration) / target_expected_duration * -1

    efficiency = float(format(efficiency, '.3f'))

    performance_score = calculate_performance_score(efficiency, rating_avg, success_rate)


    return(count, average, max, min, std, efficiency, rating_avg, success_rate, float(performance_score))


def get_performance_metrics_chart(dentist_name, treatment, df, df2):
    target_dentist = df[df['Dentist Name'] == dentist_name]
    target_treatment = target_dentist[target_dentist['Treatment'] == treatment]

    target_expected_duration = df2[df2['Treatment Menu Names'] == treatment]
    target_expected_duration = target_expected_duration['Duration'].values[0]

    # Extract the first and last dates
    start_date = df['Date'].min()
    end_date = df['Date'].max()

    # Convert month numbers to month names
    start_month = calendar.month_name[start_date.month]
    end_month = calendar.month_name[end_date.month]

    # Extract the series for plotting
    treatment_duration_series = target_treatment[['Date', 'Duration']].set_index('Date')

    return(treatment_duration_series, target_expected_duration, start_month, end_month)


def Anlyze_dentist_past_data(df_dentist_report, df_service):

    dentists = df_dentist_report['Dentist Name'].unique()

    # Ensure 'Duration' column is numeric
    df_dentist_report['Duration'] = pd.to_numeric(df_dentist_report['Duration'])

    # Calculate the average duration for each treatment per dentist
    average_duration_per_dentist_treatment = df_dentist_report.groupby(['Dentist Name', 'Treatment']).agg({'Duration': 'mean'}).reset_index()

    # Merge with the expected durations from the treatment data
    expected_durations = df_service.rename(columns={'Treatment Menu Names': 'Treatment', 'Duration': 'Expected Duration'})
    efficiency_data = average_duration_per_dentist_treatment.merge(expected_durations, on='Treatment')

    efficiency_data['Efficiency Score'] = efficiency_data.apply(calculate_efficiency_score_row, axis=1)


    # Pivot the data to get the desired format for average duration
    average_duration_df = average_duration_per_dentist_treatment.pivot(index='Dentist Name', columns='Treatment', values='Duration').reset_index()

    # Create payload1 in JSON format
    payload1 = {
        "analyzed_sheet": "average_durations",
        "df_content": []
    }

    for _, row in average_duration_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in average_duration_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload1["df_content"].append(dentist_entry)


    # Pivot the data to get the desired format for performance evaluation
    evaluation_df = efficiency_data.pivot(index='Dentist Name', columns='Treatment', values='Efficiency Score').reset_index()

    # Create payload2 in JSON format
    payload2 = {
        "analyzed_sheet": "efficiency_scores",
        "df_content": []
    }

    for _, row in evaluation_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in evaluation_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload2["df_content"].append(dentist_entry)


    # Calculate treatment score for each dentist
    performance_score_df = performance_score_sheet(df_dentist_report, evaluation_df)

    payload3 = {
        "analyzed_sheet": "performance_scores",
        "df_content": []
    }

    for _, row in performance_score_df.iterrows():
        dentist_entry = {
            "dentist_name": row['Dentist Name']
        }
        for treatment in performance_score_df.columns[1:]:  # Skipping the 'Dentist Name' column
            dentist_entry[treatment] = row[treatment]
        payload3["df_content"].append(dentist_entry)

   
    # Create the Specialty DataFrame
    specialty_data = []

    # Create the Specialty DataFrame for payload4
    payload4 = {
        "analyzed_sheet": "specialties",
        "df_content": []
    }

    for dentist in dentists:
        specialty_treatments = efficiency_data[(efficiency_data['Dentist Name'] == dentist) & (efficiency_data['Efficiency Score'] > 0.02)]
        specialties = specialty_treatments['Treatment'].tolist()
        efficiency_scores = specialty_treatments['Efficiency Score'].tolist()
        specialty_data = {
            'Dentist Name': dentist,
            'Specialty': specialties,
            'Efficiency': efficiency_scores,
            'Wage/Hour': "Edit This!"
        }
        payload4["df_content"].append(specialty_data)


    return [payload1, payload2, payload3, payload4]


df_dentist_report = pd.read_excel("./Data/Dummy_Treatment.xlsx", sheet_name="Dummy Treatments")
df_service = pd.read_excel("./Data/Service_list.xlsx", sheet_name="Main")

print(Anlyze_dentist_past_data(df_dentist_report, df_service))

({'analyzed_sheet': 'average_durations', 'df_content': [{'dentist_name': 'Arnold', 'Additional Root': 48.884434845224625, 'Advanced Filling': 24.323328302248214, 'Advanced Gum Treatment': 48.523827983882654, 'Basic Filling': 51.59288958829579, 'Basic Gum Treatment': 35.335104265758694, 'Basic Scaling': 33.694480618823285, 'Basic Tooth Extraction': 31.64343293380553, 'Basic X-ray': 3.9628158295775444, 'Consultation': 25.3135680484807, 'Dental Post & Core': 71.97060442764442, 'Dental Spa': 14.43032446965361, 'Membership Consultation Perk': 31.72642413725763, 'Mouth Guard': 36.18248189041754, 'Premium Bridge': 71.40330431321196, 'Premium Crown': 72.4197199420494, 'Prevention Seal': 16.115378415761203, 'Root Canal Treatment': 72.36980393891525, 'Wear Protect': 35.68123468639256}, {'dentist_name': 'Ashley', 'Additional Root': 59.66964134667357, 'Advanced Filling': 29.703913409878016, 'Advanced Gum Treatment': 58.90591767201521, 'Basic Filling': 66.17383284859814, 'Basic Gum Treatment': 44.9

## Run sheet scheduling V1

In [55]:
import pandas as pd
import numpy as np

TaskClassification_directory = "./Data/TaskClassification_exp.xlsx"

capabilities_df = pd.read_excel(TaskClassification_directory, sheet_name='Capabilities')
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker')
scenario_df = pd.read_excel("./data/forecast_2_weeks.xlsx")
scenario_df = scenario_df.drop(columns='Hour Required', axis=1)
service_df = pd.read_excel("./data/Service_list.xlsx")
service_df = service_df[['Treatment Menu Names', 'Duration']]
service_df.columns = ['Treatment', 'Duration']

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes

# Room Configurations
chairs_in_rooms = {
    'Room1': ['Chair1', 'Chair2'],
    'Room2': ['Chair1', 'Chair2'],
    'Room3': ['Chair3']  # Room3 is for advanced treatments, always uses Chair3
}

# Determine advanced treatments: those that Chair3 can do but Chair2 cannot
chair3_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair3'].iloc[0, 1:].astype(bool)])
chair2_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair2'].iloc[0, 1:].astype(bool)])
chair1_treatments = set(capabilities_df.columns[1:][capabilities_df.loc[capabilities_df['Object-Subject'] == 'Chair1'].iloc[0, 1:].astype(bool)])
advanced_treatments = chair3_treatments - chair2_treatments

# Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = scenario_df.melt(id_vars=['Date'], 
                                     var_name='Treatment', 
                                     value_name='Count')

# Filter out the 'Hour Required' from the forecast data
forecast_reshaped = forecast_reshaped[forecast_reshaped['Treatment'] != 'Hour Required']

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]

# Map worker types to their hourly wages and IDs
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
job_to_worker_ids = worker_df.groupby('Job')['Worker_Id'].apply(list).to_dict()

# Prepare to track worker hours
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['Date'].min()) for worker_id in worker_df['Worker_Id']}

# Update the treatment_worker_map using Worker_Id based on Job
treatment_worker_map = {}
for treatment in capabilities_df.columns[1:]:
    eligible_jobs = capabilities_df.loc[capabilities_df[treatment] == True, 'Object-Subject']
    available_workers = [worker_id for job in eligible_jobs for worker_id in job_to_worker_ids.get(job, [])]
    treatment_worker_map[treatment] = available_workers

# Prepare to store the scheduled output
output_schedule = {room: [] for room in chairs_in_rooms.keys()}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['Date'].min()} {clinic_open_time}") for room in chairs_in_rooms.keys()}

# Keep track of which room to assign the next non-advanced treatment
non_advanced_room_toggle = True

# Track the current week to reset weekly hours
current_week = pd.to_datetime(forecast_reshaped['Date'].min()).isocalendar().week

# Sort the dates to ensure they are processed in chronological order
sorted_dates = sorted(forecast_reshaped['Date'].unique())

# Iterate over each unique date in sorted order
for date in sorted_dates:
    print(f"Processing date: {date}")
    
    # Reset weekly hours if it's a new week
    week_of_year = pd.to_datetime(date).isocalendar().week
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
    
    # Reset daily hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    
    # Reset or update last end time appropriately for new day
    for worker_id in worker_last_end_time:
        worker_last_end_time[worker_id] = pd.to_datetime(date).replace(hour=0, minute=0, second=0)
    
    # Reset current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in chairs_in_rooms.keys()}
    
    # Diagnostic output to ensure reset is correct
    print(f"Worker hours reset for {date}:")
    print(f"Daily hours: {worker_hours_daily}")
    print(f"Weekly hours: {worker_hours_weekly}")
    print(f"Worker last end time: {worker_last_end_time}")
    print(f"Current time for rooms: {current_time}")
    
    # Continue with the scheduling logic
    daily_treatments = forecast_reshaped[forecast_reshaped['Date'] == date]


    # Iterate over each treatment for the day
    for index, row in daily_treatments.iterrows():
        treatment = row['Treatment']
        count = int(row['Count'])  # Number of sessions for this treatment
        
        # Get the treatment duration from the service list and introduce variability
        base_duration = service_df.loc[service_df['Treatment'] == treatment, 'Duration'].values[0]
        
        # Determine the room where the treatment is to be scheduled
        if treatment in advanced_treatments:
            room = 'Room3'
        else:
            room = 'Room1' if non_advanced_room_toggle else 'Room2'
            non_advanced_room_toggle = not non_advanced_room_toggle
        
        for i in range(count):  # Schedule each instance of the treatment sequentially
            eligible_workers = treatment_worker_map[treatment]
            selected_worker = None

            variation_factor = np.random.uniform(0.8, 1.2)  # ±20% variability
            treatment_duration = int(base_duration * variation_factor)
            
            for worker_id in eligible_workers:
                daily_available = max_working_hour_day - worker_hours_daily[worker_id]
                weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
                start_time = current_time[room]  # Start time is specific to the room
                end_time = start_time + pd.to_timedelta(treatment_duration, unit='m')
                
                # Debugging of Proposed Worker to satisfy the demands
                print(f"Worker: {worker_id}, daily: {daily_available}, Weekly: {weekly_available}, Proposed start time: {start_time}, Proposed end time: {end_time}, Treatment: {treatment}, Room: {room}")
                
                if (daily_available >= treatment_duration and
                    weekly_available >= treatment_duration and
                    worker_last_end_time[worker_id] <= start_time):
                    selected_worker = worker_id
                    worker_last_end_time[worker_id] = end_time
                    break

            if selected_worker is None:
                print(f"No worker could be selected for {treatment} on {date}. Rechecking all eligible workers.")
                for worker_id in eligible_workers:
                    start_time = current_time[room]
                    end_time = start_time + pd.to_timedelta(treatment_duration, unit='m')
                    if (worker_hours_daily[worker_id] < max_working_hour_day and
                        worker_hours_weekly[worker_id] < max_working_hour_week and
                        worker_last_end_time[worker_id] <= start_time):
                        selected_worker = worker_id
                        worker_last_end_time[worker_id] = end_time
                        break
            
            if selected_worker is None:
                raise ValueError(f"No available worker for treatment {treatment} on {date}")
        
            
            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Treatment': treatment,
                'Duration': treatment_duration,
                'Worker_Id': selected_worker,
                'Chair': 'Chair3' if room == 'Room3' else 'Chair1' if treatment in chair1_treatments else 'Chair2',
                'LaborCost': treatment_duration * (worker_wages[selected_worker] / 60)
            })
            
            # Update the current time for the room
            current_time[room] += pd.to_timedelta(treatment_duration + treatment_interval, unit='m')
            
            # Update worker hours
            worker_hours_daily[selected_worker] += treatment_duration
            worker_hours_weekly[selected_worker] += treatment_duration
            
            # Check if we exceed the clinic's operating hours for this room
            if current_time[room].time() > clinic_close_time:
                break
            
# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Add Total_Cost column for each room
for room, schedule in room_schedules.items():
    total_cost = schedule['LaborCost'].sum()
    # Add Total_Cost column with the total cost in the first row and the remaining rows empty
    schedule['Total_Cost'] = ''
    schedule.loc[0, 'Total_Cost'] = total_cost
    room_schedules[room] = schedule

# Save the output to an Excel file with separate sheets for each room
output_file_path = 'clinic_schedule_exp1.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=room, index=False)



Processing date: 2023-07-03
Worker hours reset for 2023-07-03:
Daily hours: {'D3N1': 0, 'D3N2': 0, 'D3N3': 0, 'D3N4': 0, 'D3N5': 0, 'D3N6': 0}
Weekly hours: {'D3N1': 0, 'D3N2': 0, 'D3N3': 0, 'D3N4': 0, 'D3N5': 0, 'D3N6': 0}
Worker last end time: {'D3N1': Timestamp('2023-07-03 00:00:00'), 'D3N2': Timestamp('2023-07-03 00:00:00'), 'D3N3': Timestamp('2023-07-03 00:00:00'), 'D3N4': Timestamp('2023-07-03 00:00:00'), 'D3N5': Timestamp('2023-07-03 00:00:00'), 'D3N6': Timestamp('2023-07-03 00:00:00')}
Current time for rooms: {'Room1': Timestamp('2023-07-03 08:00:00'), 'Room2': Timestamp('2023-07-03 08:00:00'), 'Room3': Timestamp('2023-07-03 08:00:00')}
Worker: D3N1, daily: 480, Weekly: 2400, Proposed start time: 2023-07-03 08:00:00, Proposed end time: 2023-07-03 08:41:00, Treatment: Basic Scaling, Room: Room1
Worker: D3N1, daily: 439, Weekly: 2359, Proposed start time: 2023-07-03 08:00:00, Proposed end time: 2023-07-03 08:18:00, Treatment: Dental Spa, Room: Room3
Worker: D3N2, daily: 480, Week

## Treatment item id JSON

In [61]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}

print(worker_treatments)


# Extract all unique treatment codes
unique_treatment_codes = set()

for treatments in worker_treatments.values():
    unique_treatment_codes.update(treatments)

# Convert the set to a list if needed
unique_treatment_codes_list = list(unique_treatment_codes)

# Function to sort treatment codes
def sort_treatment_code(treatment_code):
    # Split the treatment code into numeric and alphabetic parts
    match = re.match(r"(\d+)-([A-Z]+)", treatment_code)
    if match:
        number_part = int(match.group(1))  # Convert the numeric part to an integer
        letter_part = match.group(2)       # Get the alphabetic part
        return (number_part, letter_part)
    return (treatment_code, "")

# Sort the treatment codes
unique_treatment_codes_list.sort(key=sort_treatment_code)

print(unique_treatment_codes_list)

scenario_df = pd.DataFrame(columns=unique_treatment_codes_list)

start_date = datetime.today()
scenario_df['Date'] = [start_date + timedelta(days = i) for i in range(14)]

for item_number in unique_treatment_codes_list:
    scenario_df[item_number] = np.random.randint(0, 3, size=len(scenario_df))

print(f"sum of treatment counts in the first day: {scenario_df.loc[0, unique_treatment_codes_list].sum()}")
print(scenario_df.head())

scenario_df.to_csv("./Scenario_itemNumber2.csv", index=False)


{'OHT': ['1001-A', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '4007-A', '4008-A', '4009-A'], 'LV1': ['1001-B', '1004-D', '3014-A'], 'LV2': ['3005-B', '3015-A', '4001-A', '4002-A', '4003-A', '4005-A', '4006-A', '5007-A', '5007-B', '5008-A', '5009-A'], 'LV3': ['3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '4004-A', '5001-A', '5002-A', '5003-A', '5004-A', '5005-A', '5006-A']}
['1001-A', '1001-B', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-D', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '3005-B', '3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '3014-A', '3015-A', '4001-A', '4002-A', '4003-A', '4004-A', '4005-A', '4006-A', '4007-A', '4008-A', '4009-A', '5

## Run sheet scheduling V2

In [82]:
import json
import pandas as pd
import re

#Load data
TaskClassification_directory = "./Data/TaskClassificationV2.xlsx"
scenario_df_v2 = pd.read_csv("./Scenario_itemNumber2.csv")
constraints_df = pd.read_excel(TaskClassification_directory, sheet_name='Constraints')
worker_df = pd.read_excel(TaskClassification_directory, sheet_name='Worker')
with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

# Define the clinic conditions based on constraints
num_rooms = int(constraints_df.loc[constraints_df['Constraints'] == 'Room_num', 'Value'].values[0])
treatment_interval = int(constraints_df.loc[constraints_df['Constraints'] == 'Treatment_Interval', 'Value'].values[0])
clinic_open_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Open', 'Value'].values[0]
clinic_close_time = constraints_df.loc[constraints_df['Constraints'] == 'Clinic_Close', 'Value'].values[0]
max_working_hour_day = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Day', 'Value'].values[0]) * 60  # Convert hours to minutes
max_working_hour_week = int(constraints_df.loc[constraints_df['Constraints'] == 'Max_Working_Hour_Week', 'Value'].values[0]) * 60  # Convert hours to minutes


# Room Configurations
chairs_in_rooms = {
    'Remote': ['Chair0'],
    'Room1': ['Chair1', 'Chair2'],
    'Room2': ['Chair1', 'Chair2'],
    'Room3': ['Chair3']  # Room3 is for advanced treatments, always uses Chair3
}

# Determine each chair treatments
def get_chair_treatments(chair_number, json_data):

    chair_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['chair'] == chair_number:
                    chair_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return chair_treatments_list
                
chair1_treatments = get_chair_treatments(1, item_numbers_data)
chair2_treatments = get_chair_treatments(2, item_numbers_data)
chair3_treatments = get_chair_treatments(3, item_numbers_data)
chair0_treatments = get_chair_treatments(0, item_numbers_data)

# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}
# Reshape the forecast_2_weeks DataFrame to have a 'Treatment' and 'Count' for each date
forecast_reshaped = scenario_df_v2.melt(id_vars=['Date'], 
                                     var_name='Treatment', 
                                     value_name='Count')

# Filter out treatments with a count of 0
forecast_reshaped = forecast_reshaped[forecast_reshaped['Count'] > 0]

# Map worker types to their hourly wages and IDs
worker_wages = worker_df.set_index('Worker_Id')['Wage_Hour'].to_dict()
job_to_worker_ids = worker_df.groupby('Job')['Worker_Id'].apply(list).to_dict() #CChange the "JOB" Column format later 

# Function to get eligible treatments for each worker based on their Job
def get_eligible_treatments(worker_job, worker_treatments):
    return worker_treatments.get(worker_job, [])

# Create a new column in the DataFrame for eligible treatments
worker_df['Eligible_Treatments'] = worker_df['Job'].apply(lambda job: get_eligible_treatments(job, worker_treatments))

# Prepare to track worker hours
worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
worker_last_end_time = {worker_id: pd.to_datetime(forecast_reshaped['Date'].min()) for worker_id in worker_df['Worker_Id']}

# Prepare to store the scheduled output
output_schedule = {room: [] for room in chairs_in_rooms.keys()}

# Initialize time tracking for each room
current_time = {room: pd.to_datetime(f"{forecast_reshaped['Date'].min()} {clinic_open_time}") for room in chairs_in_rooms.keys()}

# Keep track of which room to assign the next non-advanced treatment
non_advanced_room_toggle = True

# Track the current week to reset weekly hours
current_week = pd.to_datetime(forecast_reshaped['Date'].min()).isocalendar().week

# Sort the dates to ensure they are processed in chronological order
sorted_dates = sorted(forecast_reshaped['Date'].unique())

# print(worker_df)
        
# Iterate over each unique date in sorted order
for date in sorted_dates:
    print(f"Processing date: {date}")
    
    # Reset weekly hours if it's a new week
    week_of_year = pd.to_datetime(date).isocalendar().week
    if week_of_year != current_week:
        worker_hours_weekly = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
        current_week = week_of_year
    
    # Reset daily hours for workers
    worker_hours_daily = {worker_id: 0 for worker_id in worker_df['Worker_Id']}
    
    # Reset or update last end time appropriately for new day
    for worker_id in worker_last_end_time:
        worker_last_end_time[worker_id] = pd.to_datetime(date).replace(hour=0, minute=0, second=0)
    
    # Reset current time for each room to the clinic's opening time
    current_time = {room: pd.to_datetime(f"{date} {clinic_open_time}") for room in chairs_in_rooms.keys()}
    current_time['Remote'] = pd.to_datetime(f"{date} {clinic_open_time}")  # Add Remote room
    
    # Diagnostic output to ensure reset is correct
    print(f"Worker hours reset for {date}:")
    print(f"Daily hours: {worker_hours_daily}")
    print(f"Weekly hours: {worker_hours_weekly}")
    print(f"Worker last end time: {worker_last_end_time}")
    print(f"Current time for rooms: {current_time}")
    
    # Continue with the scheduling logic
    daily_treatments = forecast_reshaped[forecast_reshaped['Date'] == date]

    for index, row in daily_treatments.iterrows():
        treatment_id = row['Treatment']
        count = int(row['Count'])  # Number of sessions for this treatment

        # Use regex to extract item_number and variation_code from treatment_id
        match = re.match(r'(\d+)-([A-Za-z]+)', treatment_id)
        if match:
            item_number = match.group(1)
            variation_code = match.group(2)
        else:
            raise ValueError(f"Invalid treatment_id format: {treatment_id}")
        
        # Initialize variables
        treatment_info = None
        variation_info = None

        # Find the corresponding treatment and variation in the JSON data
        for treatment in item_numbers_data['treatment']:
            for detail in treatment['treatment_details']:
                # Check if the item number matches
                if str(detail['item_number']) == item_number:
                    for variation in detail['variation']:
                        # Check if the variation code matches
                        if variation['variation_code'] == variation_code:
                            treatment_info = detail
                            variation_info = variation
                            break
                    if treatment_info is not None:
                        break
            if treatment_info is not None:
                break

        # If the treatment or variation info is still None, raise an error
        if treatment_info is None or variation_info is None:
            raise ValueError(f"Could not find treatment or variation information for {treatment_id}")

        # Handle treatments without procedures or with None
        procedures = variation_info.get('item_procedures', None)
        if not procedures:  # This will cover both None and empty list cases
            procedures = [{"procedure": "{-}"}]  # Set procedure to "{-}"
            procedure_duration = total_duration = float(variation_info['variation_duration'])
        else:
            total_duration = float(variation_info['variation_duration'])
            procedure_duration = total_duration / len(procedures)

        # Select the worker for this treatment
        eligible_workers = [worker_id for worker_id in worker_df['Worker_Id'] 
                            if treatment_id in worker_df.loc[worker_df['Worker_Id'] == worker_id, 'Eligible_Treatments'].values[0]]
        selected_worker = None
        
        for worker_id in eligible_workers:
            daily_available = max_working_hour_day - worker_hours_daily[worker_id]
            weekly_available = max_working_hour_week - worker_hours_weekly[worker_id]
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(procedure_duration, unit='m')
            
            # Debugging of Proposed Worker to satisfy the demands
            print(f"Worker: {worker_id}, daily: {daily_available}, Weekly: {weekly_available}, Proposed start time: {start_time}, Proposed end time: {end_time}, Treatment: {treatment_id}, Room: {room}")

            if (daily_available >= procedure_duration and
                weekly_available >= procedure_duration and
                worker_last_end_time[worker_id] <= start_time):
                selected_worker = worker_id
                worker_last_end_time[worker_id] = end_time
                break
        
        if selected_worker is None:
            raise ValueError(f"No available worker for treatment {treatment_id} on {date}")
        
        # Determine the room based on the chair
        if variation_info['chair'] == 0:
            room = 'Remote'  # Assign to Remote room
        else:
            room = 'Room3' if variation_info['chair'] == 3 else 'Room1' if non_advanced_room_toggle else 'Room2'
            non_advanced_room_toggle = not non_advanced_room_toggle

        #Labor Cost calculation each procedure
        labor_cost = total_duration * (worker_wages[selected_worker] / 60)

        # Schedule each procedure individually for the selected worker
        for procedure in procedures:
            start_time = current_time[room]
            end_time = start_time + pd.to_timedelta(procedure_duration, unit='m')

            # Schedule the treatment
            output_schedule[room].append({
                'Date': pd.to_datetime(date).date(),
                'Start_Time': start_time.strftime('%I:%M %p'),
                'Finish_Time': end_time.strftime('%I:%M %p'),
                'Item_Number': treatment_id,
                'Treatment_Names': treatment_info['item_description'],
                'Procedure': procedure['procedure'],
                'Duration': procedure_duration,
                'Worker_Id': selected_worker,
                'Chair': 'Remote' if room == 'Chair0' else 'Chair3' if room == 'Room3' else 'Chair1' if treatment_id in chair1_treatments else 'Chair2',
                'LaborCost': labor_cost
            })
            
            current_time[room] = end_time

        current_time[room] += pd.to_timedelta(treatment_interval, unit='m')
        
        # Update worker hours
        worker_hours_daily[selected_worker] += total_duration
        worker_hours_weekly[selected_worker] += total_duration
        
        # Check if we exceed the clinic's operating hours for this room
        if current_time[room].time() > clinic_close_time:
            break
            
# Convert the schedule dictionary to DataFrames for each room
room_schedules = {room: pd.DataFrame(schedule) for room, schedule in output_schedule.items()}

# Add the Total_Cost column and calculate the total labor cost for each room
for room, schedule in room_schedules.items():
    total_cost = schedule['LaborCost'].sum()
    # Add a new 'Total_Cost' column and insert the total cost in the first row
    total_cost_series = pd.Series([total_cost] + [None] * (len(schedule) - 1))
    schedule['Total_Cost'] = total_cost_series  # Add it as the last column

# Save the output to an Excel file with separate sheets for each room
output_file_path = './clinic_scheduleV2_2.xlsx'
with pd.ExcelWriter(output_file_path) as writer:
    for room, schedule in room_schedules.items():
        schedule.to_excel(writer, sheet_name=room, index=False)

Processing date: 2024-09-03 21:01:48.797914
Worker hours reset for 2024-09-03 21:01:48.797914:
Daily hours: {'N1': 0, 'N2': 0, 'OHT1': 0, 'OHT2': 0, 'OHT3': 0, 'OHT4': 0, 'OHT5': 0, 'OHT6': 0, 'OHT7': 0, 'D1N1': 0, 'D1N2': 0, 'D1N3': 0, 'D1N4': 0, 'D2N1': 0, 'D2N2': 0, 'D2N3': 0, 'D3N1': 0, 'D3N2': 0, 'D3N3': 0}
Weekly hours: {'N1': 0, 'N2': 0, 'OHT1': 0, 'OHT2': 0, 'OHT3': 0, 'OHT4': 0, 'OHT5': 0, 'OHT6': 0, 'OHT7': 0, 'D1N1': 0, 'D1N2': 0, 'D1N3': 0, 'D1N4': 0, 'D2N1': 0, 'D2N2': 0, 'D2N3': 0, 'D3N1': 0, 'D3N2': 0, 'D3N3': 0}
Worker last end time: {'N1': Timestamp('2024-09-03 00:00:00.797914'), 'N2': Timestamp('2024-09-03 00:00:00.797914'), 'OHT1': Timestamp('2024-09-03 00:00:00.797914'), 'OHT2': Timestamp('2024-09-03 00:00:00.797914'), 'OHT3': Timestamp('2024-09-03 00:00:00.797914'), 'OHT4': Timestamp('2024-09-03 00:00:00.797914'), 'OHT5': Timestamp('2024-09-03 00:00:00.797914'), 'OHT6': Timestamp('2024-09-03 00:00:00.797914'), 'OHT7': Timestamp('2024-09-03 00:00:00.797914'), 'D1N1'

## Experiment

In [60]:
import json
import re

with open("./treatment_precedence.json", "r") as file:
    item_numbers_data = json.load(file)

tretment_id_example = "1001-A"
treatment_id_procedures = [] 

# Use regex to extract item_number and variation_code from treatment_id
match = re.match(r'(\d+)-([A-Za-z]+)', tretment_id_example)
if match:
    item_number = match.group(1)
    variation_code = match.group(2)
    print(f"Item number: {item_number}, Variation code: {variation_code}")
else:
    raise ValueError(f"Invalid treatment_id format: {treatment_id}")


for treatment in item_numbers_data['treatment']:
    for detail in treatment['treatment_details']:
        if detail['item_number'] == int(item_number):
            for variation in detail['variation']:
                if variation['variation_code'] == variation_code:
                    for procedures in variation["item_procedures"]:
                        treatment_id_procedures.append(procedures['procedure'])


# Determine which worker do which treatment
def get_worker_treatments(worker_code, json_data):
    worker_treatments_list = []

    for treatment in json_data['treatment']:
        for detail in treatment['treatment_details']:
            for variation in detail['variation']:
                if variation['variation_worker_responsibility'] == worker_code:
                    worker_treatments_list.append(str(detail['item_number']) + "-" + variation['variation_code'])

    return worker_treatments_list

worker_treatments = {
    "OHT" : get_worker_treatments("OHT", item_numbers_data),
    "LV1" : get_worker_treatments("LV1", item_numbers_data),
    "LV2" : get_worker_treatments("LV2", item_numbers_data),
    "LV3" : get_worker_treatments("LV3", item_numbers_data)
}

print(worker_treatments)


# Extract all unique treatment codes
unique_treatment_codes = set()

for treatments in worker_treatments.values():
    unique_treatment_codes.update(treatments)

# Convert the set to a list if needed
unique_treatment_codes_list = list(unique_treatment_codes)

# Function to sort treatment codes
def sort_treatment_code(treatment_code):
    # Split the treatment code into numeric and alphabetic parts
    match = re.match(r"(\d+)-([A-Z]+)", treatment_code)
    if match:
        number_part = int(match.group(1))  # Convert the numeric part to an integer
        letter_part = match.group(2)       # Get the alphabetic part
        return (number_part, letter_part)
    return (treatment_code, "")

# Sort the treatment codes
unique_treatment_codes_list.sort(key=sort_treatment_code)

print(unique_treatment_codes_list)

Item number: 1001, Variation code: A
{'OHT': ['1001-A', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '4007-A', '4008-A', '4009-A'], 'LV1': ['1001-B', '1004-D', '3014-A'], 'LV2': ['3005-B', '3015-A', '4001-A', '4002-A', '4003-A', '4005-A', '4006-A', '5007-A', '5007-B', '5008-A', '5009-A'], 'LV3': ['3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '4004-A', '5001-A', '5002-A', '5003-A', '5004-A', '5005-A', '5006-A']}
['1001-A', '1001-B', '1001-C', '1002-A', '1003-A', '1003-B', '1004-A', '1004-B', '1004-C', '1004-D', '1004-E', '1005-A', '1005-B', '1005-C', '3002-A', '3002-B', '3003-A', '3003-B', '3003-C', '3004-A', '3005-A', '3005-B', '3005-C', '3005-D', '3006-A', '3007-A', '3008-A', '3009-A', '3010-A', '3011-A', '3012-A', '3013-A', '3014-A', '3015-A', '4001-A', '4002-A', '4003-A', '4004-A', '4005-A', '4006